In [3]:
!pip install plotly

In [97]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

def remove_first_0(number):
    return number[1:] if int(number[0]) == '0' else int(number)

xl = pd.read_excel(open('./xlsx/crimes-delit.xlsx', 'rb'), sheet_name='France_Entière')
effectifs_polices = pd.read_excel(open('./xlsx/effectifs_polices.xlsx', 'rb'))

cleaned_tab = effectifs_polices[effectifs_polices['MI/SG/DLPAJ/SDPA/BPA/2021'].astype(str).str.isdigit()]

In [126]:
# les 10 départements avec le plus de crimes sur les 20 derniers années
tab = []
tab_2021 = [f'_2021_{f"0{i}" if i < 10 else i}' for i in range(1, 13) ]
file = pd.ExcelFile('./xlsx/crimes-delit.xlsx')
for sheet_name in file.sheet_names:
    try:
        int(sheet_name)
    except(ValueError):
        continue
    xl = pd.read_excel(open('./xlsx/crimes-delit.xlsx', 'rb'), sheet_name=sheet_name)
    df = xl[tab_2021]
    df = df.sum(numeric_only=True).reset_index(name=sheet_name).set_index('index')
    df = df.sum(axis= 0, numeric_only=True).reset_index(name='crimes et délits').set_index('index')
    tab.append(df)
df = pd.concat(tab)
df


,crimes et délits
index,
01,27891
02,23883
03,13931
04,8006
05,6104
...,...
977,471
978,2497
986,183


In [130]:
top_criminalite = df.sort_values(by='crimes et délits', ascending=False)
fig = px.pie(top_criminalite.head(10), values='crimes et délits', names=top_criminalite.head(10).index, title='les 10 départements avec le plus haut taux de criminalité')
fig.update_traces(textposition='inside', textinfo='value+label')
fig.show()


In [31]:
sum_effectifs_polices = cleaned_tab.groupby(['MI/SG/DLPAJ/SDPA/BPA/2021'])['Unnamed: 6'].sum().reset_index(name='counts')
sum_effectifs_polices.columns = ['numéro du département', 'Nombre d’agents de police municipale']
top_effectifs_polices = sum_effectifs_polices.sort_values(by='Nombre d’agents de police municipale', ascending=False).set_index('numéro du département')

fig = px.pie(top_effectifs_polices.head(10), values='Nombre d’agents de police municipale', names=top_effectifs_polices.head(10).index, title='les 10 départements avec le plus d\'effectif de police municipal')
fig.update_traces(textposition='inside', textinfo='value+label')
fig.show()


In [91]:
nb_habitant = cleaned_tab.groupby(['MI/SG/DLPAJ/SDPA/BPA/2021'])['Unnamed: 4'].sum().reset_index(name='counts')
nb_habitant.columns = ['numéro du département', 'Nombre d\'habitant']
nb_habitant = nb_habitant.sort_values(by='Nombre d\'habitant', ascending=False).set_index('numéro du département')

fig = px.pie(nb_habitant.head(10), values='Nombre d\'habitant', names=nb_habitant.head(10).index, title='top 10 des départements avec le plus d\'habitants')
fig.update_traces(textposition='inside', textinfo='value+label')
fig.show()


In [24]:
nb_habitant_top = nb_habitant.sort_values(by='Nombre d\'habitant', ascending=False)
sum_effectifs_polices_top = sum_effectifs_polices.sort_values(by='Nombre d’agents de police municipale', ascending=False)
compare_effectifs_habitants = pd.merge(sum_effectifs_polices_top, nb_habitant_top, on="numéro du département", how="inner").set_index("numéro du département")

In [68]:
data = {}
for i in compare_effectifs_habitants.index :
    data[i] = compare_effectifs_habitants["Nombre d’agents de police municipale"][i]/compare_effectifs_habitants["Nombre d'habitant"][i]
df=pd.DataFrame(list(data.items()))
df.rename(columns = {0:"Départements", 1:"Effectif de police par rapport au nombre d'habitants"}, inplace = True)
df = df.set_index("Départements").sort_values(by="Effectif de police par rapport au nombre d'habitants", ascending=False).head(15)
df


,Effectif de police par rapport au nombre d'habitants
Départements,
987,0.001630
6,0.001267
988,0.001135
66,0.001111
83,0.001022
84,0.000979
30,0.000943
11,0.000854
971,0.000854


In [70]:

pd.options.plotting.backend = "plotly"
df.T.plot.bar(
    title="le top 15 des effectifs de police par rapport au nombre d'habitants", barmode='group')

